In [2]:
import pandas as pd
import numpy as np

def escrever(dfa, nome):
    dfa.to_excel('%s.xlsx' %nome, index=False)

# 1. Ler ficheiro com tudo e os específicos

> - **Tudo**

In [5]:
%%time
dfTudo=pd.read_csv('D:\\B&N Dados\\Delta\\AnoAnterior.csv')

Wall time: 10.3 s


> - **Específicos**

In [3]:
#dfCafe=pd.read_excel('D:\\Brands and Ninjas\\Delta\\ProdCafe.xlsx', sheet_name='Lista Lojas Sonae')
#dfLiqui=pd.read_excel('D:\\Brands and Ninjas\\Delta\\ProdLiqui.xlsx', sheet_name='Lista Lojas Sonae')

dfEspec=pd.read_excel('D:\\Brands and Ninjas\\Delta\\Mar-abril23\\ficheiro_NSS.xlsx')

- Limpar L's e seleccionar colunas de interesse

In [12]:
dfLiqui=dfLiqui[['STORE', 'STORE_NAME']]
dfLiqui['STORE'] = dfLiqui['STORE'].str.replace('L', '')
dfLiqui['STORE'] = dfLiqui['STORE'].astype('int')
dfLiqui=dfLiqui.sort_values(by="STORE")

dfCafe=dfCafe[['STORE', 'STORE_NAME']]

# 2. Passar para datetime

In [13]:
dfTudo['DATA']= pd.to_datetime(dfTudo['DATA'], format='%Y-%m-%d')
#dfEspec['DATA']= pd.to_datetime(dfEspec['DATA'], format='%Y-%m-%d')

- Pôr o Sellout bem

In [12]:
dfTudo=dfTudo.rename(columns={"VND (D-1)": "1_Dia_Antes"})

# 3. Encontrar os nomes no dataFrame de tudo

In [17]:
dfTudo['DESC_ARTIGO'].unique()

#CERV.C/ALC.T/P CORONA 35,5CL
#CERV.C/ALC.T/P FRANZISKANER WEISSBI.50CL

#CAFÉ DELTA Q QALIDUS 10CAP
#CAFÉ DELTA Q AQTIVUS 10CAP
#CAFÉ DELTA Q DEQAFEINATUS 10CAP

# Fazer subsets da dataFrame total

> - Produtos Líquidos

In [8]:
ProdLi =["CERV.C/ALC.T/P CORONA 35,5CL","CERV.C/ALC.T/P FRANZISKANER WEISSBI.50CL"]

dfTuLi = dfTudo[dfTudo['DESC_ARTIGO'].isin(ProdLi)].copy()

> - Produtos Sólidos

In [9]:
ProdCaf=["CAFÉ DELTA Q QALIDUS 10CAP","CAFÉ DELTA Q AQTIVUS 10CAP",
         "CAFÉ DELTA Q DEQAFEINATUS 10CAP"]

dfTuCaf = dfTudo[dfTudo['DESC_ARTIGO'].isin(ProdCaf)].copy()

In [49]:
dfTuLi.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 164476 entries, 4535477 to 4699952
Data columns (total 12 columns):
DATA           164476 non-null datetime64[ns]
EAN            164476 non-null int64
DESC_ARTIGO    164476 non-null object
STORE          164476 non-null int64
STORE_NAME     164476 non-null object
SOH            164476 non-null int64
INTRANSIT      164476 non-null int64
EXPECTED       164476 non-null int64
PRES_STOCK     164476 non-null int64
1_Dia_Antes    164476 non-null int64
Ciclos         164241 non-null float64
Rotura         164476 non-null int64
dtypes: datetime64[ns](1), float64(1), int64(8), object(2)
memory usage: 16.3+ MB


# Mergir

In [13]:
def mergir(dfa, dfb):
    dfNome=pd.merge(dfa, dfb, how="right", on = "STORE")
    
    
    colunasBem = {col: col.replace('_x', '') for col in dfNome.columns if col.endswith('_x')}
    dfNome = dfNome.rename(columns=colunasBem)
    colunasMal = [col for col in dfNome.columns if col.endswith('_y')]
    dfNome = dfNome.drop(colunasMal, axis=1)
    return dfNome


> - Produtos Líquidos

In [14]:

dfLiqFinal=mergir(dfTuLi,dfEspec)


> - Produtos Sólidos

In [15]:

dfCafFinal=mergir(dfTuCaf,dfEspec)


> - Tudo

In [16]:

dfJuntos = pd.concat([dfCafFinal,dfLiqFinal], ignore_index=True)



In [14]:
produtos=["CAFÉ DELTA Q QALIDUS 10CAP","CAFÉ DELTA Q AQTIVUS 10CAP",
         "CAFÉ DELTA Q DEQAFEINATUS 10CAP","CERV.C/ALC.T/P CORONA 35,5CL","CERV.C/ALC.T/P FRANZISKANER WEISSBI.50CL"]
dfJuntos =dfTudo[dfTudo['DESC_ARTIGO'].isin(produtos)].copy()

In [15]:
dfJuntos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 416806 entries, 1636886 to 4699952
Data columns (total 12 columns):
DATA           416806 non-null datetime64[ns]
EAN            416806 non-null int64
DESC_ARTIGO    416806 non-null object
STORE          416806 non-null int64
STORE_NAME     416806 non-null object
SOH            416806 non-null int64
INTRANSIT      416806 non-null int64
EXPECTED       416806 non-null int64
PRES_STOCK     416806 non-null int64
1_Dia_Antes    416806 non-null int64
Ciclos         416571 non-null float64
Rotura         416806 non-null int64
dtypes: datetime64[ns](1), float64(1), int64(8), object(2)
memory usage: 41.3+ MB


In [18]:
colunasExtra=["Hora", "Nome da Loja"]

dfJuntos = dfJuntos.drop(colunasExtra, axis=1)

# 4. Colunas dias anteriores

In [20]:
dfJuntos['Dia_Actual'] = dfJuntos['1_Dia_Antes'].shift(-1) #todas as colunas sobem (dia seguinte)

In [65]:
dfJuntosOrg=dfJuntos.iloc[:, [0,1,2,3,4,5,6,7,8,9]]

In [69]:
dfJuntosOrg.head()

,DATA,EAN,DESC_ARTIGO,STORE,STORE_NAME,SOH,INTRANSIT,EXPECTED,PRES_STOCK,Rotura,...,1_Dia_Antes,2_Dia_Antes,3_Dia_Antes,4_Dia_Antes,5_Dia_Antes,6_Dia_Antes,7_Dia_Antes,8_Dia_Antes,9_Dia_Antes,10_Dia_Antes
0,2022-01-01,5601082026978,CAFÉ DELTA Q AQTIVUS 10CAP,1,CNT MATOSINHOS,288,0,0,96,0,...,14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-01-02,5601082026978,CAFÉ DELTA Q AQTIVUS 10CAP,1,CNT MATOSINHOS,288,0,0,96,0,...,0,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-01-03,5601082026978,CAFÉ DELTA Q AQTIVUS 10CAP,1,CNT MATOSINHOS,274,0,0,96,0,...,14,0.0,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-01-04,5601082026978,CAFÉ DELTA Q AQTIVUS 10CAP,1,CNT MATOSINHOS,264,0,0,96,0,...,10,14.0,0.0,14.0,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-01-05,5601082026978,CAFÉ DELTA Q AQTIVUS 10CAP,1,CNT MATOSINHOS,249,0,120,96,0,...,15,10.0,14.0,0.0,14.0,NaN,NaN,NaN,NaN,NaN


In [67]:
def dias(df, dia):         #dia é quantos dias antes
    a=int(dia)-1
    valores = df.loc[:,'1_Dia_Antes'].shift(a)
    valores[:a] = np.nan
    df.loc[:,'%s_Dia_Antes' % dia] = valores

In [68]:
dia1=2
diaFinal=10
dfFinal=dfJuntosOrg.copy()
for i in range(dia1, diaFinal+1):
    dias(dfFinal, i)

# 5. Fins de Semana

In [47]:
dfFdsSell = dfJuntos[dfJuntos['DATA'].dt.weekday.isin([4,5,6])]         #Sexta, Sábado e Domingo
dfSemanaSell = dfJuntos[dfJuntos['DATA'].dt.weekday.isin([0,1,2,3])]    #Segunda a Quinta

# 6. Escrever

> - Manipular antes

In [23]:
dfMarAbr = dfJuntos.loc[(pivoted_df['DATA'] >= '2022-04-08') & (dfJuntos['DATA'] <= '2022-04-10')]

dfMarAbrFds = dfMarAbr[dfMarAbr['DATA'].dt.weekday.isin([4, 5, 6])]

In [159]:
pivoted_df = dfMarAbr.pivot_table(index=['DATA','STORE'], columns='DESC_ARTIGO', values='Dia_Actual')
pivoted_df = pivoted_df.reset_index().rename_axis(None, axis=1)
pivoted_df = pivoted_df.rename(columns={'CAFÉ DELTA Q AQTIVUS 10CAP': 'SELLOUT CAFÉ DELTA Q AQTIVUS 10CAP 22', 
                                  "CAFÉ DELTA Q DEQAFEINATUS 10CAP":"SELLOUT CAFÉ DELTA Q DEQAFEINATUS 10CAP 22",
                                 "CAFÉ DELTA Q QALIDUS 10CAP":"SELLOUT CAFÉ DELTA Q QALIDUS 10CAP 22",
                                 "CERV.C/ALC.T/P FRANZISKANER WEISSBI.50CL":"SELLOUT CERV.C/ALC.T/P FRANZISKANER WEISSBI.50CL 22",
                                 "CERV.C/ALC.T/P CORONA 35,5CL":"SELLOUT CERV.C/ALC.T/P CORONA 35,5CL 22"})

In [160]:
pivoted_df['DATA'] = pivoted_df['DATA'].replace({
    pd.to_datetime('2022-04-08'): pd.to_datetime('2023-03-31'),
    pd.to_datetime('2022-04-09'): pd.to_datetime('2023-04-01'),
    pd.to_datetime('2022-04-10'): pd.to_datetime('2023-04-02')
})

In [161]:
pivoted_df['DATA'] = pd.to_datetime(pivoted_df['DATA'])
pivoted_df["DATA"].unique()
pivoted_df=pivoted_df.sort_values(by=["STORE","DATA"])

In [127]:
produtos=pivoted_df.columns[2:]

In [162]:
for i in produtos:
    for j in (0,len(pivoted_df.index)/3-1):
        a=int(j)*3
        pivoted_df[i].iat[a]=(pivoted_df[i].iat[a]+pivoted_df[i].iat[a+1]+pivoted_df[i].iat[a+2])/3



In [165]:
nova=pivoted_df.loc[(pivoted_df['DATA'] == '2023-03-31')]


In [167]:
nova=nova.drop(columns=['DATA'])
nova.head()

,STORE,SELLOUT CAFÉ DELTA Q AQTIVUS 10CAP 22,SELLOUT CAFÉ DELTA Q DEQAFEINATUS 10CAP 22,SELLOUT CAFÉ DELTA Q QALIDUS 10CAP 22,"SELLOUT CERV.C/ALC.T/P CORONA 35,5CL 22",SELLOUT CERV.C/ALC.T/P FRANZISKANER WEISSBI.50CL 22
0,1,4.0,4.0,9.333333,10.666667,7.0
1,2,34.0,8.0,22.000000,0.000000,2.0
2,4,18.0,6.0,3.000000,26.000000,2.0
3,5,14.0,4.0,7.000000,0.000000,2.0
4,6,38.0,9.0,17.000000,0.000000,12.0


In [74]:
dfMarAbrFds = dfMarAbr[dfMarAbr['DATA'].dt.weekday.isin([4, 5, 6])]

In [1]:
def fds(nome, df, numero):
    a = int(numero)
    b = 3*(a-1)
    nome=df[b:b+3].copy()
    return nome


In [128]:
quantos=5
for i in range(1, quantos):
    fds("fds%s" %i, dfMarAbrFds, i)

fds1=fds("fds1", dfMarAbrFds, 1)
fds2=fds("fds2", dfMarAbrFds, 2)
fds3=fds("fds3", dfMarAbrFds, 3)
fds4=fds("fds4", dfMarAbrFds, 4)
fds5=fds("fds5", dfMarAbrFds, 5)

In [78]:
fds1= dfMarAbrFds.loc[(dfMarAbrFds['DATA'] >= '2022-03-04') & (dfMarAbrFds['DATA'] <= '2022-03-06')]
fds2= dfMarAbrFds.loc[(dfMarAbrFds['DATA'] >= '2022-03-11') & (dfMarAbrFds['DATA'] <= '2022-03-13')]
fds3= dfMarAbrFds.loc[(dfMarAbrFds['DATA'] >= '2022-03-18') & (dfMarAbrFds['DATA'] <= '2022-03-20')]
fds4= dfMarAbrFds.loc[(dfMarAbrFds['DATA'] >= '2022-03-25') & (dfMarAbrFds['DATA'] <= '2022-03-27')]
fds5= dfMarAbrFds.loc[(dfMarAbrFds['DATA'] >= '2022-04-08') & (dfMarAbrFds['DATA'] <= '2022-04-10')]

> - Agora pode ser

In [71]:

escrever(fds1, "fds1")
escrever(fds2, "fds2")
escrever(fds3, "fds3")               #sexta a domingo
escrever(fds4, "fds4")
escrever(fds5, "fds5")



NameError: name 'fds1' is not defined

In [168]:
#escrever(pivoted_df, "sellout0422")
#escrever(nova, "sellmed")


In [70]:
%%time
# Dias de Semana

escrever(dfFds, 'ProdutosLíquidos2022fds')
escrever(dfSemana, 'ProdutosSólidos2022semana')


Wall time: 2min 11s
